In [1]:
import sys
import os 

path = '../src/'
path = os.path.normcase(path)
if not any(os.path.normcase(sp) == path for sp in sys.path):
    sys.path.append(path)

In [2]:
text = """
    [ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ 
						[ recipient1, recipient2 ]
					]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]
 """

In [7]:
import ast
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory

In [8]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ [ recipient1, recipient2 ] ]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]



In [9]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

source = Location.resolve_from_text('Walmart')
destination = Location.resolve_from_text('Walgreens')
directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time)
Responder.respond(response=directions)
recipient1 = Contact.resolve_from_text('Pepsi')
recipient2 = Contact.resolve_from_text('Walgreens')
for recipient in [recipient1, recipient2]:
    exact_content = Content.resolve_from_text('check it out')
    Message.send_message(recipient=recipient, exact_content=exact_content)
